In [12]:
import pickle
from bs4 import BeautifulSoup
import requests
import urllib
import re
import time
from random import randint
from multiprocessing import Process, Queue

In [ ]:
with open('id_data_0-14974.pkl', 'r') as picklefile:
    id_data = pickle.load(picklefile)

In [2]:
with open('book_data_0-14974.pkl', 'r') as picklefile:
    book_data = pickle.load(picklefile)

In [7]:
sorted_book_data = sorted(book_data.keys(), key=lambda x: int(x))

In [24]:
sorted_book_data = sorted_book_data[66850 :]

In [28]:
sorted_book_data = sorted_book_data[55000:]

In [44]:
test_list = sorted_book_data[:5]

In [45]:
get_full_book_data(test_list)

19186417
19186417 done
19186421
19186421 done
19188154
19188154 done
19189533
19189533 done
19189811


/Users/David/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))
Process Worker-16:
Process Worker-13:
Process Worker-15:
Process Worker-14:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/David/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/David/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/David/anaconda/lib/python2.7/multiprocessing/process.py", l

KeyboardInterrupt: 

In [39]:
def main():
    queue = Queue()
    for num in sorted_book_data:
        queue.put(num)
    num_workers = 4
    queue.put(None)
    workers = []
    print 'workers created'
    for i in range(num_workers):
        worker = Worker(queue)
        workers.append(worker)
    print 'workers started'
    for w in workers:
        w.start()

workers created
workers started


In [40]:


class Worker(Process):
    '''
    process to download data from goodreads book pages using queue numbers
    '''
    def __init__(self, queue):
        '''
        initializing worker. setting queue variable.
        '''
        Process.__init__(self)
        self._queue = queue
    def run(self):
        '''
        running process. downloading and saving data.
        '''
        while True:
            
            lst = []
            for i in xrange(1000):
                item = self._queue.get()
                if item == None:
                    self._queue.put(None)
                    break
                else:
                    lst.append(self._queue.get())
            if len(lst) == 0:
                break
            
            get_full_book_data(lst)
        

In [42]:
def check_for_title(soup):
    if soup.findAll(id="bookTitle"):
        return True
    else: 
        return False

def check_for_cover(soup):
    if soup.findAll(class_="noCoverMediumImage"):
        return False
    else: 
        return True
    
def get_top_shelves(url_extension):
    url_base = 'http://goodreads.com'
    shelves_url = url_base + url_extension
    try: 
        response = requests.get(shelves_url)
    except: 
        time.sleep(60)
        response = requests.get(shelves_url)
    soup = BeautifulSoup(response.text)
    key_word_dict = {}
    for item in soup.findAll(class_="shelfStat"):
        keyword = item.findAll(class_="mediumText actionLinkLite")[0].text
        num_people = item.findAll(class_="smallText")[0].text
        num = re.findall(r'\d+', num_people.replace(",", ''))[0]
        key_word_dict[keyword] = num
    return key_word_dict

def get_full_book_data(book_id_list):    
    full_book_data = {}
    for key in book_id_list:
        ##for sanity:
        print key
        full_book_data[key]={}
        url = 'http://www.goodreads.com/book/show/{}'.format(str(key)+'.')
        response = requests.get(url)
        soup = BeautifulSoup(response.text)
        if check_for_title(soup) == True:
            title = soup.findAll(id="bookTitle")[0].text.strip(' \n',)
        else: 
            continue
        if check_for_cover(soup) == True:
            image_url = soup.findAll(id="coverImage")[0]['src']
        else:
            continue
        author = soup.findChildren(class_="authorName")[0].text
        try:
            description = soup.findChildren(id="description")[0].contents[3].text
        except IndexError:
            description = 'none'
        full_book_data[key]['title'] = title
        full_book_data[key]['author'] = author
        full_book_data[key]['description'] = description
        full_book_data[key]['users'] = book_data[key]
        full_book_data[key]['image_url'] = image_url
        urllib.urlretrieve(image_url, "/Users/David/Desktop/images1/{}.jpg".format(key))
        ##get keywords for book and how many users associated them with the book
        try:
            url_extension = soup.findAll('a', href=True, text='See top shelves...')[0]['href']
        except IndexError:
            if key in full_book_data:
                del full_book_data[key]
                continue
        keywords_dict = get_top_shelves(url_extension)
        full_book_data[key]['keywords'] = keywords_dict
        print str(key) + ' done'
    print 'writing to pickle!***********************'
    beginning = sorted(full_book_data.keys(), key=lambda x: int(x))[0]
    end = sorted(full_book_data.keys(), key=lambda x: int(x))[-1]
    with open('full_book_data_{}-{}.pkl'.format(beginning, end), 'w') as picklefile:
        pickle.dump(full_book_data, picklefile)

In [6]:
len(sorted_book_data[])

126308

In [ ]:
full_book_data = {}
len(full_book_data)


In [ ]:
sorted()

In [34]:
def check_for_title(soup):
    if soup.findAll(id="bookTitle"):
        return True
    else: 
        return False

def check_for_cover(soup):
    if soup.findAll(class_="noCoverMediumImage"):
        return False
    else: 
        return True
    
def get_top_shelves(url_extension):
    url_base = 'http://goodreads.com'
    shelves_url = url_base + url_extension
    try: 
        response = requests.get(shelves_url)
    except: 
        time.sleep(60)
        response = requests.get(shelves_url)
    soup = BeautifulSoup(response.text)
    key_word_dict = {}
    for item in soup.findAll(class_="shelfStat"):
        keyword = item.findAll(class_="mediumText actionLinkLite")[0].text
        num_people = item.findAll(class_="smallText")[0].text
        num = re.findall(r'\d+', num_people.replace(",", ''))[0]
        key_word_dict[keyword] = num
    return key_word_dict


def get_full_book_data(book_data):
    count = -1
    #for key in sorted(book_data.keys())[start_point:]:
    '''if len(full_book_data) > 0:
        last_processed = full_book_data.keys()[-1]
        index_start_position = list(book_data.keys()).index(last_processed)
    else:
        index_start_position = 0'''
        
    if len(full_book_data) != 0:
        ## find the last processed item in the book dictionary so as to pick up where we left off
        ## in case of an interruption
        last_processed = sorted(full_book_data.keys(), key=lambda x: int(x))[-1]
        ## the index of the last processed item, in the original book_data dictionary
        ## and continue from there, which is where we left off.
        index_start_position = sorted(book_data.keys(), key=lambda x: int(x)).index(last_processed)
        print str(index_start_position)
    else:
        index_start_position = 60000
    for key in sorted(book_data.keys(), key=lambda x: int(x))[index_start_position:]:
        ##for sanity:
        print key
        full_book_data[key]={}
        url = 'http://www.goodreads.com/book/show/{}'.format(str(key)+'.')
        response = requests.get(url)
        soup = BeautifulSoup(response.text)
        if check_for_title(soup) == True:
            title = soup.findAll(id="bookTitle")[0].text.strip(' \n',)
        else: 
            continue
        if check_for_cover(soup) == True:
            image_url = soup.findAll(id="coverImage")[0]['src']
        else:
            continue
        author = soup.findChildren(class_="authorName")[0].text
        try:
            description = soup.findChildren(id="description")[0].contents[3].text
        except IndexError:
            description = 'none'
        full_book_data[key]['title'] = title
        full_book_data[key]['author'] = author
        full_book_data[key]['description'] = description
        full_book_data[key]['users'] = book_data[key]
        full_book_data[key]['image_url'] = image_url
        urllib.urlretrieve(image_url, "/Users/David/Desktop/images/{}.jpg".format(key))
        ##get keywords for book and how many users associated them with the book
        try:
            url_extension = soup.findAll('a', href=True, text='See top shelves...')[0]['href']
        except IndexError:
            if key in full_book_data:
                del full_book_data[key]
                continue
        keywords_dict = get_top_shelves(url_extension)
        full_book_data[key]['keywords'] = keywords_dict
        print str(key) + ' done'
        count += 1
        if count % 1000 == 0:
            print 'writing to pickle!***********************'
            beginning = sorted(book_data.keys(), key=lambda x: int(x))[0]
            end = sorted(book_data.keys(), key=lambda x: int(x))[-1]
            with open('full_book_data_{}-{}.pkl'.format(beginning, end), 'w') as picklefile:
                pickle.dump(full_book_data, picklefile)

In [ ]:
for i in range(1, 100):
    try:
        get_full_book_data(book_data)
    except requests.ConnectionError:
        print 'connection error ________________'
        continue

In [ ]:
with open('full_book_data_{}-{}.pkl'.format(60001,66849), 'w') as picklefile:
    pickle.dump(full_book_data, picklefile)

In [ ]:
sorted(book_data.keys(), key=lambda x: int(x)).index(first_processed)

In [ ]:
first_processed = sorted(full_book_data.keys(), key=lambda x: int(x))[0]
first_processed


In [ ]:
last_processed = sorted(full_book_data.keys(), key=lambda x: int(x))[-1]
last_processed

In [ ]:
sorted(book_data.keys(), key=lambda x: int(x)).index(last_processed)

In [ ]:
len(sorted(full_book_data.keys(), key=lambda x: int(x)))

In [ ]:
index_start_position = sorted(book_data.keys(), key=lambda x: int(x)).index(last_processed)

In [ ]:
sorted(book_data.keys(), key=lambda x: int(x))[27]

In [ ]:
index_start_position

In [ ]:
sorted(book_data.keys(), key=lambda x: int(x))[:10]

In [ ]:
last_processed = sorted(full_book_data.keys()[-1], key=lambda x: int(x))
index_start_position = sorted(book_data.keys()[:10], key=lambda x: int(x)).index(last_processed)

In [ ]:
def get_top_shelves(url_extension):
    url_base = 'http://goodreads.com'
    shelves_url = url_base + url_extension
    try: 
        response = requests.get(shelves_url)
    except: 
        time.sleep(60)
        response = requests.get(shelves_url)
    soup = BeautifulSoup(response.text)
    key_word_dict = {}
    for item in soup.findAll(class_="shelfStat"):
        keyword = item.findAll(class_="mediumText actionLinkLite")[0].text
        num_people = item.findAll(class_="smallText")[0].text
        num = re.findall(r'\d+', num_people.replace(",", ''))[0]
        key_word_dict[keyword] = num
    return key_word_dict
        

In [ ]:
#url = 'http://www.goodreads.com/book/show/{}'.format(3876.)
url = 'http://www.goodreads.com/work/shelves/589497'
response = requests.get(url)
soup = BeautifulSoup(response.text)

In [ ]:
soup.findAll('a', href=True, text='See top shelves...')[0]['href']

In [33]:
def check_for_cover(soup):
    if soup.findAll(class_="noCoverMediumImage"):
        return False
    else: 
        return True

    lst.append(self._queue.get())
    item = self._queue.get()
  File "/Users/David/anaconda/lib/python2.7/multiprocessing/queues.py", line 117, in get
  File "/Users/David/anaconda/lib/python2.7/multiprocessing/queues.py", line 115, in get
    res = self._recv()
    self._rlock.acquire()
KeyboardInterrupt
KeyboardInterrupt


In [ ]:
def check_for_title(soup):
    if soup.findAll(id="bookTitle"):
        return True
    else: 
        return False

In [ ]:
soup.findAll(class_="shelfStat")[1].findAll(class_="mediumText actionLinkLite")[0].text

In [ ]:
num_people = soup.findAll(class_="shelfStat")[1].findAll(class_="smallText")[0].text
num = re.findall(r'\d+', num_people.replace(",", ''))[0]
num

In [ ]:
soup.findAll(class_="shelfStat")[1].findAll(class_="smallText")[0].text.strip(",")

In [ ]:
key_word_dict = {}
for item in soup.findAll(class_="shelfStat"):
    keyword = item.findAll(class_="mediumText actionLinkLite")[0].text
    num_people = item.findAll(class_="smallText")[0].text
    num = re.findall(r'\d+', num_people.replace(",", ''))[0]
    key_word_dict[keyword] = num